## 6.8 长短期记忆 (LSTM)

### 6.8.1 长短期记忆

> **引入3个门,输入门(input gate),遗忘门(forget gate),输出门(output gate),以及与隐藏状态相同的记忆细胞**

#### 6.8.1.1 输入门,遗忘门和输出门

> 1. **门的输入均为当前时间步输入$\boldsymbol{X}_{t}$与上一时间步隐藏状态$\boldsymbol{H}_{t-1}$;输出有激活函数sigmoid的全连接,因此值域均为$[0, 1]$**
> 2. **假设隐藏单元个数为$h$,时间步$t$的小批量输入$\boldsymbol{X}_{t} \in \mathbb{R}^{n \times d}$(样本数为$n$,输入个数为$d$)和上一时间步隐藏状态$\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h}$**
> 3. **时间步$t$输入门$\boldsymbol{I}_{t} \in \mathbb{R}^{n \times h}$,遗忘门$\boldsymbol{F}_{t} \in \mathbb{R}^{n \times h}$,输出门$\boldsymbol{O}_{t} \in \mathbb{R}^{n \times h}$计算如下**

$$ \begin{aligned} \boldsymbol{I}_{t} = \sigma(\boldsymbol{X}_{t}\boldsymbol{W}_{xi} + \boldsymbol{H}_{t-1}\boldsymbol{W}_{hi} + \boldsymbol{b}_{i})  \end{aligned} $$
$$ \begin{aligned} \boldsymbol{F}_{t} = \sigma(\boldsymbol{X}_{t}\boldsymbol{W}_{xf} + \boldsymbol{H}_{t-1}\boldsymbol{W}_{hf} + \boldsymbol{b}_{f})  \end{aligned} $$
$$ \begin{aligned} \boldsymbol{O}_{t} = \sigma(\boldsymbol{X}_{t}\boldsymbol{W}_{xo} + \boldsymbol{H}_{t-1}\boldsymbol{W}_{ho} + \boldsymbol{b}_{o})  \end{aligned} $$

> **$\boldsymbol{W}_{xi},\boldsymbol{W}_{xf},\boldsymbol{W}_{xo} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hi},\boldsymbol{W}_{hf},\boldsymbol{W}_{ho} \in \mathbb{R}^{h \times h}$是权重参数,$\boldsymbol{b}_{i},\boldsymbol{b}_{f},\boldsymbol{b}_{o}$是偏差参数**


![长短期记忆中输入门,遗忘门和输出门计算](img/6.8_lstm_0.svg)

#### 6.8.1.2 候选记忆细胞

> 1. **记忆细胞$\tilde{\boldsymbol{C}_{t}} \in \mathbb{R}^{n \times h}$的计算和3个门类似,使用值域为$[-1, 1]$的tanh作为激活函数**
$$\begin{aligned} \tilde{\boldsymbol{C}_{t}} = \text{tanh}(\boldsymbol{X}_{t}\boldsymbol{W}_{xc} + \boldsymbol{H}_{t-1}\boldsymbol{W}_{hc} + \boldsymbol{b}_{c}) \end{aligned}$$

> **其中$\boldsymbol{W}_{xc} \in \mathbb{R}^{d \times h}, \boldsymbol{W}_{hc} \in \mathbb{R}^{h \times h}$是权重参数,$\boldsymbol{b}_{c} \in \mathbb{R}^{1 \times h}$是偏差参数**

![长短期记忆中候选记忆细胞的计算](img/6.8_lstm_1.svg)

#### 6.8.1.3 记忆细胞

> 1. **通过值域$[0, 1]$的输入门,遗忘门,输出门来控制隐藏状态中信息的流动**
> 2. **当前$\boldsymbol{C}$的计算组合了上一时间步记忆细胞和当前候选细胞**
$$ \begin{aligned} \boldsymbol{C}_{t} = \boldsymbol{F}_{t} \odot \boldsymbol{C}_{t-1} + \boldsymbol{I}_{t} \odot \tilde{\boldsymbol{C}_{t}}\end{aligned}$$

> 3. **遗忘门控制上一时间步记忆细胞是否传递到当前时间步,输入门控制当前输入$\boldsymbol{X}_{t}$通过候选细胞$\tilde{\boldsymbol{C}_{t}}$如何流入当前时间步的记忆细胞**
> 4. **如何遗忘门一直近似1,输入门一直近似0,则过去的时间不讲一直通过时间保存并传递到当前时间步,可以应对`RNN`中梯度衰减问题**


![长短期记忆中记忆细胞的计算](img/6.8_lstm_2.svg)

#### 6.8.1.4 隐藏状态

> 1. **可以通过输出门来控制从记忆细胞到隐藏状态$\boldsymbol{H}_{t} \in \mathbb{R}^{n \times h}$**
$$\begin{aligned} \boldsymbol{H}_{t} = \boldsymbol{O}_{t} \odot \text{tanh}(\boldsymbol{C}_{t})\end{aligned}$$
> 2. **$tanh$确保隐藏状态值域$[-1, 1]$**
>     - **输出门近似为1,记忆细胞信息将传递到隐藏状态供输出层使用**
>     - **输出门近似为0,记忆细胞信息只自己保留**

![长短期记忆中隐藏太的计算](img/6.8_lstm_3.svg)

### 6.8.2 读取数据集

In [9]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch.utils as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

### 6.8.3 从零开始实现

#### 6.8.3.1 初始化模型参数

In [17]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size


def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
               _one((num_hiddens, num_hiddens)),
               torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数
    
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

### 6.8.4 定义模型

In [18]:
def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device),
           torch.zeros((batch_size, num_hiddens), device=device))

In [19]:
# 只有隐藏太会传递到输出层,记忆细胞不参与

In [20]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)
# def lstm(inputs, state, params):
#     [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
#     (H, C) = state
#     outputs = []
#     for X in inputs:
#         I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
#         F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
#         O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
#         C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
#         C = F * C + I * C_tilda
#         H = O * C.tanh()
#         Y = torch.matmul(H, W_hq) + b_q
#         outputs.append(Y)
#     return outputs, (H, C)

#### 6.8.4.1 训练模型并创造歌词

In [21]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [22]:
d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 214.975842, time 1.32 sec
 - 分开 我不不的我 我不你的我 我不你的我 我不你的我 我不你的我 我不你的我 我不你的我 我不你的我 我
 - 不分开 我不不的我 我不你的我 我不你的我 我不你的我 我不你的我 我不你的我 我不你的我 我不你的我 我
epoch 80, perplexity 66.589328, time 1.83 sec
 - 分开 我想你这你 我不要这你 我不要这样 我不要这生 我不要这生 我不要这生 我不要这生 我不要这生 我
 - 不分开 我想你这你 我不要这你 我不要这样 我不要这生 我不要这生 我不要这生 我不要这生 我不要这生 我
epoch 120, perplexity 14.709074, time 1.40 sec
 - 分开 一直我 一小两 在九么 在九的碗有 一制盘 一片了 装一了 装一段 一片段 装一了 装一段 装一段
 - 不分开你的可不美 漂坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我
epoch 160, perplexity 3.795056, time 1.51 sec
 - 分开 一个是 是谁一碗热 配上上斤 漫谁黄沙 白色蜡烛 温暖了空 全人的梦 全人放纵 没有有纵 全人己纵
 - 不分开你的没样 我要道你 我不要再想  没有我的生活 我知好你生活 不知歌觉 又你了一个秋 后知后觉 我该


### 6.8.5 简洁实现

In [8]:
lr = 1e-2
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(lstm_layer, vocab_size)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.020666, time 1.02 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开的玩笑 想通 却又再考倒我 说散 你想很久了吧? 败给你的黑色幽默 不想太多 我想一定是我听错弄错搞
epoch 80, perplexity 1.011446, time 1.07 sec
 - 分开始移动 在感当 我想大声宣布 对你依依不舍 连隔壁邻居都猜到我现在的感受 河边的风 在吹着头发飘动 
 - 不分开 我有一双翅膀 二双翅膀 随时出发 偷偷出发 我一定带我妈走  从前的教育别人的家庭  别人的爸爸种
epoch 120, perplexity 1.009252, time 1.00 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开 我真的看不下去 以为我较细汉 从小到大只有妈妈的温暖  为什么我爸爸 那么凶 如果真的我有一双翅膀
epoch 160, perplexity 1.009643, time 0.99 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 习武之人切记 仁者无敌 是谁在练太极 风生水起 快
